In [ ]:
import argparse
import json 
import cv2

import os
from os import listdir, walk
from os.path import isfile, join
from tqdm import tqdm

import fiftyone as fo
import fiftyone.zoo as foz

import matplotlib.pyplot as plt
import numpy as np

def read_json(data):
    with open(data) as f:
        return json.load(f)
    
def get_list_of_files_in_folder(path):
    list_of_files = []
    for (dirpath, dirnames, filenames) in walk(path):
        list_of_files.extend(filenames)
        break
    return list_of_files

def check_key(instance, key):
    if key in instance:
        return instance[key]
    else:
        return True
    
def fix_poly(poly, shape):
    poly = [ (min(max(0, point[0]), shape[0]), min(max(0, point[1]), shape[1])) for point in poly]
    return poly

def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

def restore_poly(poly, width, height):
    poly_new = []
    for i in poly:
        poly_new.append([int(i[0] * width), int(i[1] * height)])
    return poly_new
    
#python converterCocoToVoxel.py -c '/home/fishial/Fishial/dataset/export/03_export_Verified_ALL.json.json' -i '/home/fishial/Fishial/dataset/fishial_collection/data' -ds 'export-fishial-november-test'

In [ ]:

COCO_PATH = r'/home/fishial/Fishial/dataset/export_07_09_2023/06_export_Verified_ALL.json'
IMAGE_FOLDER = "/home/fishial/Fishial/dataset/export_14_06_2023/DATA/data"
DATASET_NAME = "fishial-05-09-2023_v0.6"

data_coco_export = read_json(COCO_PATH)

categories = {}
for k in data_coco_export['categories']:
    categories.update({k['id']: {
        'name':k['name'],
        'supercategory':k['supercategory'],
        'species_id':k['fishial_extra']['species_id']
    }})

list_of_files = get_list_of_files_in_folder(IMAGE_FOLDER)

images_infos = {}
for inst_id, image_inst in enumerate(data_coco_export['images']):
    print(f"Left: {inst_id}/{len(data_coco_export['images'])} count: {len(images_infos)}", end='\r')
    file_name = image_inst['file_name']
    if file_name not in list_of_files: continue
    if 'fishial_extra' not in image_inst: continue
    if check_key(image_inst['fishial_extra'], 'xray'): continue
    if check_key(image_inst['fishial_extra'], 'not_a_real_fish'): continue
    if check_key(image_inst['fishial_extra'], 'no_fish'): continue
    if check_key(image_inst['fishial_extra'], 'test_image'): continue

    try:
        image = cv2.imread(os.path.join(IMAGE_FOLDER, file_name))
    except Exception as e:
        continue
    height, width = image.shape[:2]
    images_infos.update({
        image_inst['id']: {
            'file_name': os.path.join(IMAGE_FOLDER, file_name),
            'width': width,
            'height': height,
            'include_in_odm': image_inst['fishial_extra']['include_in_odm']
        }
    })

annotations_infos = {}
for ann_id, ann_inst in enumerate(data_coco_export['annotations']):
    print(f"Left: {ann_id}/{len(data_coco_export['annotations'])}", end='\r')

    image_id = ann_inst['image_id']

    try:
        drawn_fish_id = ann_inst['fishial_extra']['drawn_fish_id']

        poly = [(
            int(ann_inst['segmentation'][0][point * 2]), 
            int(ann_inst['segmentation'][0][point * 2 + 1])) for point in range(int(len(ann_inst['segmentation'][0])/2))]
        poly = fix_poly(poly, [images_infos[image_id]['width'], images_infos[image_id]['height']])
        poly = [[points[0]/images_infos[image_id]['width'], points[1]/images_infos[image_id]['height']] for points in poly]
    except Exception as e:
        continue
    ann_id = ann_inst['id']
    category_id = ann_inst['category_id']
    if 'poly' in images_infos[image_id]:
        images_infos[image_id]['poly'].append([categories[category_id], drawn_fish_id, ann_id, poly])
    else:
        images_infos[image_id].update({
        'poly': [[categories[category_id], drawn_fish_id, ann_id, poly]]
    }) 

In [ ]:

samples = []

for idx, image_id in enumerate(images_infos):
    print(f"Left: {idx}/{len(images_infos)}", end='\r')
    if 'poly' not in images_infos[image_id]: continue
    if len(images_infos[image_id]['poly']) == 0: continue 

    dict_of_labels = {}
    for poly in images_infos[image_id]['poly']:
        category, drawn_fish_id, ann_id, polylines = poly
        
        polyline = fo.Polyline(
                label=category['supercategory'],
                points=[polylines],
                closed=False,
                filled=True)
       
        polyline['species_id'] = category['species_id']
        polyline['ann_id'] = ann_id
        polyline['area'] = PolyArea(np.array(polylines)[:,0], np.array(polylines)[:,1])
        polyline['drawn_fish_id'] = drawn_fish_id
        
        if category['name'] in dict_of_labels:
            dict_of_labels[category['name']].append(polyline)
        else:
            dict_of_labels.update({category['name']: [polyline]})
    odm_tag = 'include_in_odm' if images_infos[image_id]['include_in_odm'] else 'not_include_in_odm'
    sample = fo.Sample(filepath=images_infos[image_id]['file_name'], tags=[odm_tag])
    for label in dict_of_labels:
        sample[label] = fo.Polylines(polylines=dict_of_labels[label])
    sample["image_id"] = image_id
    sample["width"] = images_infos[image_id]['width']
    sample["height"] = images_infos[image_id]['height']
    

    samples.append(sample)

dataset = fo.Dataset(DATASET_NAME)
dataset.add_samples(samples)
dataset.persistent = True
print("Finish")

In [ ]:
for i in data_coco_export['annotations']:
    if i['image_id'] == 45:
        print(i)

In [ ]:
dataset = fo.Dataset('fishial-full-14-06-2023')
dataset.add_samples(samples)
dataset.persistent = True
print("Finish")

In [ ]:
sample = images_infos[245109]
file_name = sample['file_name']
image = cv2.imread(file_name)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
width = sample['width']
height = sample['height']
for poly in images_infos[245109]['poly']:
    #print(poly[3])
    poly_restored = restore_poly(poly[3], width, height)
    pts = np.array(poly_restored, np.int32)
    pts = pts.reshape((-1, 1, 2))
    isClosed = True
    color = (255, 0, 0)
    thickness = 5
    image = cv2.polylines(image, [pts],
                          isClosed, color, thickness)
plt.imshow(image)
plt.show()

In [ ]:

DATASET_NAME_OLD = "2023.08.11.13.45.36"
dataset_new = fo.load_dataset(DATASET_NAME)
dataset_old = fo.load_dataset(DATASET_NAME_OLD)

In [ ]:
dict_tags = {}
for sample_old in tqdm(dataset_old):
    dict_tags.update({sample_old['image_id']: sample_old['tags'] })

In [ ]:
for sample_new in tqdm(dataset_new):
    if str(sample_new['image_id']) in dict_tags:
        sample_new['tags'].extend(dict_tags[str(sample_new['image_id'])])
        sample_new['tags'] = list(set(sample_new['tags']))
    else:
        sample_new['tags'].append("train")
    sample_new.save()